# Gradio让你的机器学习模型性感起来

gradio是一个快速构建机器学习Web展示页面的开源Python库。

只需要几行代码，就可以让你的机器学习模型从抽象晦涩的代码变成性感可爱的交互界面。

让没有任何编程技能的用户也能够轻松使用和体验模型。

它非常适合在模型迭代测试中快速获取用户反馈或者在汇报展示中进行使用，非常酷炫。

相比另一个机器学习应用web展示库streamlit，gradio具有如下优势:

* 便于分享：gradio可以在启动应用时设置share=True参数创建外部分享链接，可以直接在微信中分享给用户使用。

* 方便调试：gradio可以在jupyter中直接展示页面，更加方便调试。



大多数的gradio应用一般由如下最常用的基础模块构成。

* 应用界面：gr.Interface(简易场景), gr.Blocks(定制化场景)

* 输入输出：gr.Image(图像), gr.Textbox(文本框), gr.DataFrame(数据框), gr.Dropdown(下拉选项), gr.Number(数字), gr.Markdown, gr.Files

* 控制组件：gr.Button(按钮)

* 布局组件：gr.Tab(标签页), gr.Row(行布局), gr.Column(列布局)






我们将由易到难通过5个范例来介绍gradio的使用方法。

* hello world范例 （gr.Interface）

* 文本分类 （gr.Interface）

* 图片分类 （gr.Interface）

* 目标检测  （gr.Blocks定制化）

* 图片筛选器 （gr.Blocks）

参考资料：
* 官方教程：https://gradio.app/ 

* B站视频演示: https://www.bilibili.com/video/BV1BN411w7Ub/




## 一，Hello World (难度系数: ⭐️)

In [5]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
gr.close_all()
demo.launch(share=True)


/opt/anaconda3/envs/deeplearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: /opt/anaconda3/envs/deeplearning/lib/python3.11/site-packages/gradio/frpc_darwin_arm64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.2
3. Move the file to this location: /opt/anaconda3/envs/deeplearning/lib/python3.11/site-packages/gradio


## 二，文本分类 (难度系数: ⭐️⭐️)

In [7]:
#!pip install gradio, ultralytics, transformers, torchkeras 

In [19]:
import gradio as gr 
from transformers import pipeline

pipe = pipeline("text-classification")

def clf(text):
    result = pipe(text)
    label = result[0]['label']
    score = result[0]['score']
    res = {label:score,'POSITIVE' if label=='NEGATIVE' else 'NEGATIVE': 1-score}
    return res 

demo = gr.Interface(fn=clf, inputs="text", outputs="label")
gr.close_all()
demo.launch(share=True)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /distilbert/distilbert-base-uncased-finetuned-sst-2-english/resolve/714eb0f/config.json (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))"), '(Request ID: b12e1209-e4af-4fa8-8c17-054dcaf49d98)')

## 三，图片分类 (难度系数: ⭐️⭐️⭐)

In [21]:
import gradio as gr 
import pandas as pd 
from ultralytics import YOLO
from skimage import data
from PIL import Image

model = YOLO('yolov8n-cls.pt')
#prepare example image 
Image.fromarray(data.coffee()).save('coffee.jpeg') 
Image.fromarray(data.astronaut()).save('people.jpeg')
Image.fromarray(data.cat()).save('cat.jpeg')
def predict(img):
    result = model.predict(source=img)
    df = pd.Series(result[0].names).to_frame()
    df.columns = ['names']
    df['probs'] = result[0].probs
    df = df.sort_values('probs',ascending=False)
    res = dict(zip(df['names'],df['probs']))
    return res
gr.close_all() 
demo = gr.Interface(fn = predict,inputs = gr.Image(type='pil'), outputs = gr.Label(num_top_classes=5), 
                    examples = ['cat.jpeg','people.jpeg','coffee.jpeg'])
demo.launch()



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/sherlock/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


ModuleNotFoundError: No module named 'skimage'

## 四，目标检测 (难度系数: ⭐️⭐️⭐⭐️)

In [ ]:
import gradio as gr 
import pandas as pd 
from skimage import data
from PIL import Image
from torchkeras import plots 
from torchkeras.data import get_url_img
from pathlib import Path
from ultralytics import YOLO
import ultralytics
from ultralytics.yolo.data import utils 

model = YOLO('yolov8n.pt')

#prepare example images
Image.fromarray(data.coffee()).save('coffee.jpeg') 
Image.fromarray(data.astronaut()).save('people.jpeg')
Image.fromarray(data.cat()).save('cat.jpeg')

#load class_names
yaml_path = str(Path(ultralytics.__file__).parent/'datasets/coco128.yaml') 
class_names = utils.yaml_load(yaml_path)['names']

In [ ]:
def detect(img):
    if isinstance(img,str):
        img = get_url_img(img) if img.startswith('http') else Image.open(img).convert('RGB')
    result = model.predict(source=img)
    if len(result[0].boxes.boxes)>0:
        vis = plots.plot_detection(img,boxes=result[0].boxes.boxes,
                     class_names=class_names, min_score=0.2)
    else:
        vis = img
    return vis

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# yolov8目标检测演示")

    with gr.Tab("捕捉摄像头喔"):
        in_img = gr.Image(source='webcam',type='pil')
        button = gr.Button("执行检测",variant="primary")

        gr.Markdown("## 预测输出")
        out_img = gr.Image(type='pil')

        button.click(detect,
                     inputs=in_img, 
                     outputs=out_img)
        
    
    with gr.Tab("选择测试图片"):
        files = ['people.jpeg','coffee.jpeg','cat.jpeg']
        drop_down = gr.Dropdown(choices=files,value=files[0])
        button = gr.Button("执行检测",variant="primary")
        
        
        gr.Markdown("## 预测输出")
        out_img = gr.Image(type='pil')
        
        button.click(detect,
                     inputs=drop_down, 
                     outputs=out_img)
        
    with gr.Tab("输入图片链接"):
        default_url = 'https://t7.baidu.com/it/u=3601447414,1764260638&fm=193&f=GIF'
        url = gr.Textbox(value=default_url)
        button = gr.Button("执行检测",variant="primary")
        
        gr.Markdown("## 预测输出")
        out_img = gr.Image(type='pil')
        
        button.click(detect,
                     inputs=url, 
                     outputs=out_img)
        
    with gr.Tab("上传本地图片"):
        input_img = gr.Image(type='pil')
        button = gr.Button("执行检测",variant="primary")
        
        gr.Markdown("## 预测输出")
        out_img = gr.Image(type='pil')
        
        button.click(detect,
                     inputs=input_img, 
                     outputs=out_img)
        

        
        

In [ ]:
gr.close_all() 
demo.queue(concurrency_count=5)
demo.launch()


## 五，图片筛选器 (难度系数: ⭐️⭐️⭐⭐️⭐️)

尽管gradio的设计初衷是为了快速创建机器学习用户交互页面。

但实际上，通过组合gradio的各种组件，用户可以很方便地实现非常实用的各种应用小工具。

例如:  数据分析展示dashboard,  数据标注工具, 制作一个小游戏界面等等。

本范例我们将应用 gradio来构建一个图片筛选器，从百度爬取的一堆猫咪表情包中刷选一些我们喜欢的出来。



In [ ]:
#!pip install -U torchkeras

In [ ]:
import torchkeras 
from importlib import reload 
reload(torchkeras)
from torchkeras.data import download_baidu_pictures 
download_baidu_pictures('猫咪表情包',100)

In [ ]:
import gradio as gr
from PIL import Image
import time,os
from pathlib import Path 
base_dir = '猫咪表情包'
selected_dir = 'selected'
files = [str(x) for x in 
         Path(base_dir).rglob('*.jp*g') 
         if 'checkpoint' not in str(x)]
def show_img(path):
    return Image.open(path)
def fn_before(done,todo):
    if done>=1:
        done = done-1
        todo = todo+1
        
    else:
        done = done
        todo = todo
    
    path = files[int(done-1)]
    img = show_img(path)
    
    return done,todo,path,img
def fn_next(done,todo):
    if todo>=1:
        done = done+1
        todo = todo-1
    else:
        done = done
        todo = todo
        
    path = files[int(done-1)]
    img = show_img(path)

    return done,todo,path,img
def save_selected(img_path):
    img_name = os.path.basename(img_path)
    if img_path.startswith('http'):
        img = get_image(img_path).convert('RGB')
        img_path = 'tmp.jpg'
        img.save(img_path)
    selected_files = set(os.listdir(selected_dir))
    
    msg = ''
    if img_name not in selected_files:
        save_path = os.path.join(selected_dir,img_name)
        img = Image.open(img_path)
        img.save(save_path)
        msg = 'selected images number = {}\n'.format(len(selected_files)+1)+\
              'Save image sucessed!\n'+\
              'Saved image name : {}'.format(img_name)
    else:
        msg = 'selected images number = {}\n'.format(len(selected_files))+\
        "Don't save duplicate images!"
    return msg 
def get_default_msg():
    if not os.path.exists(selected_dir):
        os.mkdir(selected_dir)
    selected_files = set(os.listdir(selected_dir))
    msg = 'Selected images number = {}\n'.format(len(selected_files))
    return msg 

In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        total = gr.Number(len(files),label='总数量')
        with gr.Row(scale = 1):
            bn_before = gr.Button("上一张")
            bn_next = gr.Button("下一张")
        with gr.Row(scale = 2):
            done = gr.Number(0,label='已完成')
            todo = gr.Number(len(files),label='待完成')
    path = gr.Text(files[0],lines=1, label='当前图片路径')
    feedback_button = gr.Button("选择图片",variant="primary")
    msg = gr.TextArea(value=get_default_msg,lines=3,max_lines = 5)
    img = gr.Image(value = show_img(files[0]),type='pil')
    
    bn_before.click(fn_before,
                 inputs= [done,todo], 
                 outputs=[done,todo,path,img])
    bn_next.click(fn_next,
                 inputs= [done,todo], 
                 outputs=[done,todo,path,img])
    feedback_button.click(save_selected,
                         inputs = path,
                         outputs = msg
                         )

In [ ]:
 demo.launch()

## 六，huggingface托管 (难度系数: ⭐️⭐️)

为了便于向合作伙伴永久展示我们的模型App,可以将gradio的模型部署到 HuggingFace的 Space托管空间中，完全免费的哦。

方法如下：

1，注册huggingface账号：https://huggingface.co/join

2，在space空间中创建项目：https://huggingface.co/spaces

3，创建好的项目有一个Readme文档，可以根据说明操作，也可以手工编辑app.py和requirements.txt文件。

参考范例。

https://huggingface.co/spaces/lyhue1991/yolov8_demo

![算法美食屋logo.png](https://tva1.sinaimg.cn/large/e6c9d24egy1h41m2zugguj20k00b9q46.jpg)